In [1]:
import opendatasets as od
dataset_url = 'https://www.kaggle.com/daveianhickey/2000-16-traffic-flow-england-scotland-wales'
od.download(dataset_url)

Skipping, found downloaded files in "./2000-16-traffic-flow-england-scotland-wales" (use force=True to force download)


In [2]:
import pandas as pd
import json
import geopandas as gpd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv(r'2000-16-traffic-flow-england-scotland-wales/accidents_2012_to_2014.csv')

<ipython-input-3-ba28ce43a273>:1: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'2000-16-traffic-flow-england-scotland-wales/accidents_2012_to_2014.csv')


In [4]:
df.columns.values

array(['Accident_Index', 'Location_Easting_OSGR',
       'Location_Northing_OSGR', 'Longitude', 'Latitude', 'Police_Force',
       'Accident_Severity', 'Number_of_Vehicles', 'Number_of_Casualties',
       'Date', 'Day_of_Week', 'Time', 'Local_Authority_(District)',
       'Local_Authority_(Highway)', '1st_Road_Class', '1st_Road_Number',
       'Road_Type', 'Speed_limit', 'Junction_Detail', 'Junction_Control',
       '2nd_Road_Class', '2nd_Road_Number',
       'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions',
       'Weather_Conditions', 'Road_Surface_Conditions',
       'Special_Conditions_at_Site', 'Carriageway_Hazards',
       'Urban_or_Rural_Area',
       'Did_Police_Officer_Attend_Scene_of_Accident',
       'LSOA_of_Accident_Location', 'Year'], dtype=object)

In [5]:
df['Local_Authority_(District)'].unique()

array([ 12,   1,  11,  28, 570,   3,   5,   4,   8,   2,   9,  30,  31,
        27,  25,  17,  24,  57,  26,  13,  14,  16,  15,  20,  19,  10,
        22,   7,   6,  18,  29,  32,  23,  21,  64,  62,  65,  60,  61,
        63,  74,  71,  85,  76,  79,  73,  83,  70,  84,  75,  80,  82,
        72,  77,  93,  91,  90,  92,  95, 102, 107, 110, 109, 100, 114,
       112, 101, 106, 104, 130, 129, 128, 124, 139, 150, 148, 147, 149,
       146, 169, 161, 185, 189, 180, 182, 184, 186, 187, 181, 204, 203,
       200, 206, 202, 211, 213, 215, 210, 233, 232, 231, 228, 240, 241,
       243, 245, 309, 307, 303, 305, 306, 302, 300, 255, 252, 258, 256,
       253, 250, 251, 257, 254, 277, 276, 273, 270, 285, 274, 286, 278,
       284, 293, 294, 290, 291, 292, 329, 325, 327, 320, 322, 321, 328,
       324, 323, 344, 340, 345, 341, 346, 342, 347, 343, 354, 350, 351,
       355, 356, 353, 352, 360, 366, 364, 362, 368, 363, 361, 367, 365,
       386, 380, 382, 383, 381, 385, 384, 394, 395, 393, 392, 39

In [6]:
with open('2000-16-traffic-flow-england-scotland-wales/Local_Authority_Districts_Dec_2016.geojson','r') as file:
    localAuthorities = json.load(file)

In [7]:
localAuthoritiesMap = {}
for feature in localAuthorities['features']:
    localAuthoritiesMap[feature['properties']['objectid']] = feature['properties']['lad16nm']

In [8]:
for key, value in localAuthoritiesMap.items():
    if value == "Reading":
        print(key, value)
    if value == "Swindon":
        print(key, value)

30 Swindon
38 Reading


In [9]:
readingCrash = df.loc[(df['Latitude'] <= 51.481774) & (df['Longitude'] >= -1.063511) & (df['Latitude'] >= 51.427912) & (df['Longitude'] <= -0.909006)]


In [10]:
readingCrash

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year
130848,201243P049012,474660,176240,-0.926275,51.480146,43,2,2,3,07/01/2012,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Wet/Damp,None,None,2,No,E01028660,2012
132402,201243R043112,473450,172460,-0.944481,51.446322,43,3,2,1,01/11/2012,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Wet/Damp,None,None,1,Yes,E01016418,2012
132630,201243R100052,471800,173710,-0.967966,51.457771,43,3,2,1,10/05/2012,...,non-junction pedestrian crossing,Daylight: Street light present,Raining without high winds,Wet/Damp,None,None,1,Yes,E01016351,2012
132960,201243R183112,475260,171760,-0.918589,51.439791,43,3,2,1,07/11/2012,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Wet/Damp,Roadworks,Pedestrian in carriageway (not injured),1,Yes,E01016668,2012
133974,201243W002102,472890,172130,-0.952605,51.443427,43,3,2,1,01/10/2012,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,E01016370,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420528,201443W297124,471288,174215,-0.975233,51.462376,43,1,3,3,27/12/2014,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01016355,2014
420529,201443W298114,472961,171750,-0.951663,51.440002,43,3,2,1,28/11/2014,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01016370,2014
420530,201443W300124,465707,176212,-1.055187,51.481005,43,3,3,2,15/12/2014,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Wet/Damp,None,None,1,Yes,E01016317,2014
420537,201443W307124,471976,173668,-0.965443,51.457371,43,3,2,1,19/12/2014,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01033418,2014


In [11]:
readingCrash.count()

Accident_Index                                 1108
Location_Easting_OSGR                          1108
Location_Northing_OSGR                         1108
Longitude                                      1108
Latitude                                       1108
Police_Force                                   1108
Accident_Severity                              1108
Number_of_Vehicles                             1108
Number_of_Casualties                           1108
Date                                           1108
Day_of_Week                                    1108
Time                                           1108
Local_Authority_(District)                     1108
Local_Authority_(Highway)                      1108
1st_Road_Class                                 1108
1st_Road_Number                                1108
Road_Type                                      1108
Speed_limit                                    1108
Junction_Detail                                   0
Junction_Con

In [12]:
readingCrash['Local_Authority_(District)'].unique()

array([484, 472, 475, 471])

In [13]:
readingCrash.groupby(['Local_Authority_(District)']).count()


,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year
Local_Authority_(District),,,,,,,,,,,,,,,,,,,,,
471,63,63,63,63,63,63,63,63,63,63,...,63,63,63,63,63,63,63,63,63,63
472,933,933,933,933,933,933,933,933,933,933,...,933,933,933,933,933,933,933,933,930,933
475,108,108,108,108,108,108,108,108,108,108,...,108,108,108,108,108,108,108,108,108,108
484,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [14]:
readingCrash = df.loc[df['Local_Authority_(District)'] == 472]

In [15]:
readingCrash

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location,Year
132397,201243R042112,471490,168970,-0.973383,51.415196,43,3,2,2,01/11/2012,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Wet/Damp,None,None,2,Yes,E01016439,2012
132402,201243R043112,473450,172460,-0.944481,51.446322,43,3,2,1,01/11/2012,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Wet/Damp,None,None,1,Yes,E01016418,2012
132630,201243R100052,471800,173710,-0.967966,51.457771,43,3,2,1,10/05/2012,...,non-junction pedestrian crossing,Daylight: Street light present,Raining without high winds,Wet/Damp,None,None,1,Yes,E01016351,2012
133974,201243W002102,472890,172130,-0.952605,51.443427,43,3,2,1,01/10/2012,...,No physical crossing within 50 meters,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,E01016370,2012
133976,201243W003012,471190,174350,-0.976616,51.463602,43,3,2,2,01/01/2012,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,No,E01016355,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420527,201443W296124,473069,172492,-0.949956,51.446659,43,2,1,1,28/12/2014,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01016418,2014
420528,201443W297124,471288,174215,-0.975233,51.462376,43,1,3,3,27/12/2014,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01016355,2014
420529,201443W298114,472961,171750,-0.951663,51.440002,43,3,2,1,28/11/2014,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01016370,2014
420537,201443W307124,471976,173668,-0.965443,51.457371,43,3,2,1,19/12/2014,...,No physical crossing within 50 meters,Darkness: Street lights present and lit,Fine without high winds,Dry,None,None,1,Yes,E01033418,2014


In [16]:
y = df['Number_of_Casualties']
y

0         1
1         1
2         1
3         1
4         1
         ..
464692    1
464693    2
464694    1
464695    2
464696    2
Name: Number_of_Casualties, Length: 464697, dtype: int64

In [17]:
X = df[['Accident_Severity','Speed_limit', 'Weather_Conditions']]
X = pd.get_dummies(data=X, drop_first=True)
#readingCrash[readingCrash.columns.difference(['Number_of_Casualties'])]
X

,Accident_Severity,Speed_limit,Weather_Conditions_Fine without high winds,Weather_Conditions_Fog or mist,Weather_Conditions_Other,Weather_Conditions_Raining with high winds,Weather_Conditions_Raining without high winds,Weather_Conditions_Snowing with high winds,Weather_Conditions_Snowing without high winds,Weather_Conditions_Unknown
0,3,30,1,0,0,0,0,0,0,0
1,3,30,1,0,0,0,0,0,0,0
2,3,30,1,0,0,0,0,0,0,0
3,3,30,1,0,0,0,0,0,0,0
4,3,30,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
464692,2,70,0,0,0,0,0,0,1,0
464693,3,70,1,0,0,0,0,0,0,0
464694,3,40,1,0,0,0,0,0,0,0
464695,2,60,0,0,0,0,1,0,0,0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
regression = LinearRegression().fit(X_train, y_train)

In [20]:
regression.score(X_test, y_test)

0.0236321734576328

In [21]:
correctPredictions = y_test == regression.predict(X_test).round()
correctPredictions.sum()/len(correctPredictions)

0.7240774432511037

In [22]:
y_test

372239    1
104062    1
242078    1
101329    1
187822    2
         ..
301718    1
441788    1
122825    1
153351    1
88879     1
Name: Number_of_Casualties, Length: 153351, dtype: int64

In [27]:
def predict(inputs):
    return [1] * len(inputs)

In [28]:
correctPredictions = y_test == predict(X_test) 
correctPredictions.sum()/len(correctPredictions)

0.7765779160227191